# TFM Suministro Global - Países - Neo4j - py2neo 

## Partimos de la existencia, en paralelo a esta ejecución, de una instancia levantada y accesible de Neo4j  
### La primera vez que se quiera usar _py2neo_ erá  necesario asegurar su instalación:  _!pip install py2neo_

In [1]:
##!pip install py2neo 

In [2]:
#importar la librería para poder hacer uso de sus funcionalidades y objetos
from py2neo import *

In [3]:
#acceder a la instancia levantada de Neo4j - en nuestro caso instancia Docker - asignamos credenciales de autentificación -
graph_test01 = Graph(auth=('neo4j', 's3cr3t'))

In [4]:
#que es el objeto - graph_test01
graph_test01

Graph('bolt://localhost:7687')

In [5]:
#que es el objeto - tx
tx = graph_test01.begin()

In [6]:
tx

In [22]:
#query de conexión de prueba sin errores
# verificar existencia de NODOS previos de MATERIALES Brutos y Refinados
#--- cuenta los nodos con diferentes tipos de etiqueta y los une en una única salida, 1 fila por cada tipo (name, cnt)
queryPRB= "MATCH (n)                                    \
RETURN labels (n)[0] as name, count (n) as Ntotal       \
UNION MATCH (n)                                         \
WITH labels (n)[1] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
"
graph_test01.run(queryPRB)

name,Ntotal
MaterialB,78
MaterialREF,78


## CARGAR DATOS

In [1]:
import numpy as np    #datos y operativa álgegra y matrices de n dimensiones (arrays)
import pandas as pd   #carga dataset y manipulación de datos  y dataframes

In [ ]:
### DATOS: Fichero completo de PAISES en el suministro crítico GLOBAL - Atendiendo al alcance del estudio - EU CRM 2020
### FICHERO:GLPaises_cargar.csv
**CONTENIDO: Países con alguna producción global crítica **
USO: Crear los NODOS de tipo "Pais"  
 -- propiedades de cada pais: nombre (name) y Pais_iso3 --> con el código internacional ISO de 3 caracteres alfanuméricos   
  -- los datos de suministro -posteriores- fijarán las relaciones entre países proveedores y sus materiales producidos  
rutaFichero =  "https://raw.githubusercontent.com/JuanKFD/UNIR_TFM/main/GLPaises_cargar.csv" 
dfPaises = pd.read_csv(rutaFichero, sep=';')


In [61]:
dfPaises.head()

,Pais,Pais_iso3
0,Algeria,DZA
1,Albania,ALB
2,Argentina,ARG
3,Armenia,ARM
4,Australia,AUS


In [62]:
dfPaises.tail()

,Pais,Pais_iso3
86,Russian Federation,RUS
87,Rwanda,RWA
88,Saudi Arabia,SAU
89,Senegal,SEN
90,Sierra Leone,SLE


In [63]:
dfPaises.shape

(91, 2)

In [64]:
dfPaises.columns

Index(['Pais', 'Pais_iso3'], dtype='object')

In [65]:
nFilas = dfPaises.shape[0]

In [66]:
nFilas

91

### Cargados en memoria los _91 países_ de la cadena de suministro global

# PREVISUALIZACION DE DATOS - PAISES


In [67]:
for i in range(0, nFilas):
    print(dfPaises.Pais[i])
    print(dfPaises.Pais_iso3[i])


Algeria
DZA
Albania
ALB
Argentina
ARG
Armenia
ARM
Australia
AUS
Austria
AUT
Bahrain
BHR
Bangladesh
BGD
Belarus
BLR
Belgium
BEL
Bolivia
BOL
Brazil
BRA
Bulgaria
BGR
Burkina Faso
BFA
Burundi
BDI
Canada
CAN
Chile
CHL
China
CHN
Colombia
COL
Congo, D.R.
COD
Cote D_Ivoire
CIV
Cuba
CUB
Czech Republic
CZE
Denmark
DNK
Dominican Republic
DOM
Ecuador
ECU
Egypt, Arab Rep.
EGY
Estonia
EST
Ethiopia
ETH
Finland
FIN
France
FRA
Gabon
GAB
Georgia
GEO
Germany
DEU
Ghana
GHA
Greece
GRC
Guatemala
GTM
Guinea
GIN
Guyana
GUY
Hong Kong
HKG
Hungary
HUN
Iceland
ISL
India
IND
Indonesia
IDN
Iran, Islamic Rep.
IRN
Iraq
IRQ
Ireland
IRL
Israel
ISR
Italy
ITA
Jamaica
JAM
Japan
JPN
Jordan
JOR
Kazakhstan
KAZ
Kenya
KEN
Korea, Dem. Rep.
PRK
Korea, Rep.
KOR
Kuwait
KWT
Kyrgyz Republic
KGZ
Lao Pdr
LAO
Lithuania
LTU
Liberia
LBR
Madagascar
MDG
Malaysia
MYS
Mali
MLI
Mexico
MEX
Mongolia
MNG
Morocco
MAR
Mozambique
MOZ
Myanmar
MMR
Namibia
NAM
Netherlands
NLD
New Caledonia
NCL
Nigeria
NGA
Norway
NOR
Oman
OMN
Others
OTHS
Other EU
OTHEU

In [68]:
for colName in dfPaises.columns:
    print(colName)

Pais
Pais_iso3


In [69]:
dfPaises.columns[1]

'Pais_iso3'

In [70]:
#  Crear NODO DE PAIS por cada fila junto con sus PROPIEDADES
#  Nodos: PAIS -- para países productores de algún material de cadena global de suministro crítica
## PREVISAULIZAR QUERYs antes de la ejecución
TipoNodo = " :Pais " 
for i in range(0, nFilas):
    strQry = "CREATE (p " + TipoNodo                                             \
        + "{"                                                                    \
        + "name: '" + dfPaises.Pais[i] + "', "                                   \
        + dfPaises.columns[1] + ": " + "'" + dfPaises.Pais_iso3[i]               \
                + "' })"                                                         \
        + " RETURN p" 
    print("QRY [" + str(i) + "]> "+ strQry)
# querys  visualizadas

QRY [0]> CREATE (p  :Pais {name: 'Algeria', Pais_iso3: 'DZA' }) RETURN p
QRY [1]> CREATE (p  :Pais {name: 'Albania', Pais_iso3: 'ALB' }) RETURN p
QRY [2]> CREATE (p  :Pais {name: 'Argentina', Pais_iso3: 'ARG' }) RETURN p
QRY [3]> CREATE (p  :Pais {name: 'Armenia', Pais_iso3: 'ARM' }) RETURN p
QRY [4]> CREATE (p  :Pais {name: 'Australia', Pais_iso3: 'AUS' }) RETURN p
QRY [5]> CREATE (p  :Pais {name: 'Austria', Pais_iso3: 'AUT' }) RETURN p
QRY [6]> CREATE (p  :Pais {name: 'Bahrain', Pais_iso3: 'BHR' }) RETURN p
QRY [7]> CREATE (p  :Pais {name: 'Bangladesh', Pais_iso3: 'BGD' }) RETURN p
QRY [8]> CREATE (p  :Pais {name: 'Belarus', Pais_iso3: 'BLR' }) RETURN p
QRY [9]> CREATE (p  :Pais {name: 'Belgium', Pais_iso3: 'BEL' }) RETURN p
QRY [10]> CREATE (p  :Pais {name: 'Bolivia', Pais_iso3: 'BOL' }) RETURN p
QRY [11]> CREATE (p  :Pais {name: 'Brazil', Pais_iso3: 'BRA' }) RETURN p
QRY [12]> CREATE (p  :Pais {name: 'Bulgaria', Pais_iso3: 'BGR' }) RETURN p
QRY [13]> CREATE (p  :Pais {name: 'Burkin

In [72]:
#comprobar que sigue abierta la conexión para enviar las queries a la BD 
graph_test01

Graph('bolt://localhost:7687')

In [73]:
# Crear un nodo por cada fila con sus propiedades
#nodos de PAIS,
# CREAR NODOS: "Pais" 
## query para crea nodo con sus  propiedades
TipoNodo = " :Pais " 
for i in range(0, nFilas):
    strQry = "CREATE (p " + TipoNodo                                             \
        + "{"                                                                    \
        + "name: '" + dfPaises.Pais[i] + "', "                                   \
        + dfPaises.columns[1] + ": " + "'" + dfPaises.Pais_iso3[i]               \
                + "' })"                                                         \
        + " RETURN p" 
#ejecutar Query en BD abierta - Neo4j
    graph_test01.run(strQry)
#Visualizar Query ejecutada 
    print("EJECUTADA QRY [" + str(i) + "]> "+ strQry)
# nodos creados

EJECUTADA QRY [0]> CREATE (p  :Pais {name: 'Algeria', Pais_iso3: 'DZA' }) RETURN p
EJECUTADA QRY [1]> CREATE (p  :Pais {name: 'Albania', Pais_iso3: 'ALB' }) RETURN p
EJECUTADA QRY [2]> CREATE (p  :Pais {name: 'Argentina', Pais_iso3: 'ARG' }) RETURN p
EJECUTADA QRY [3]> CREATE (p  :Pais {name: 'Armenia', Pais_iso3: 'ARM' }) RETURN p
EJECUTADA QRY [4]> CREATE (p  :Pais {name: 'Australia', Pais_iso3: 'AUS' }) RETURN p
EJECUTADA QRY [5]> CREATE (p  :Pais {name: 'Austria', Pais_iso3: 'AUT' }) RETURN p
EJECUTADA QRY [6]> CREATE (p  :Pais {name: 'Bahrain', Pais_iso3: 'BHR' }) RETURN p
EJECUTADA QRY [7]> CREATE (p  :Pais {name: 'Bangladesh', Pais_iso3: 'BGD' }) RETURN p
EJECUTADA QRY [8]> CREATE (p  :Pais {name: 'Belarus', Pais_iso3: 'BLR' }) RETURN p
EJECUTADA QRY [9]> CREATE (p  :Pais {name: 'Belgium', Pais_iso3: 'BEL' }) RETURN p
EJECUTADA QRY [10]> CREATE (p  :Pais {name: 'Bolivia', Pais_iso3: 'BOL' }) RETURN p
EJECUTADA QRY [11]> CREATE (p  :Pais {name: 'Brazil', Pais_iso3: 'BRA' }) RETUR

In [74]:
dfGraphMat= graph_test01.run("MATCH (n :Pais ) RETURN n").data()

In [75]:
dfGraphMat[0]


{'n': Node('Pais', Pais_iso3='DZA', name='Algeria')}

In [79]:
#validamos que todos los -nuevos- nodos se han creado :Pais  
graph_test01.run("MATCH (n :Pais) RETURN  LABELS(n), COUNT(n)")

LABELS(n),COUNT(n)
['Pais'],91


In [80]:
#validamos que todos los nodos se han creado (hasta este paso)
queryPRB= "MATCH (n)                                    \
RETURN labels (n)[0] as name, count (n) as Ntotal       \
UNION MATCH (n)                                         \
WITH labels (n)[1] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
UNION MATCH (n)                                         \
WITH labels (n)[2] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN  name, Ntotal                                    \
"
graph_test01.run(queryPRB)


name,Ntotal
MaterialB,78
MaterialREF,78
Pais,91


# fin proceso - PAISes- CREADOS

In [81]:
# fin archivo